In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.shell import sqlContext
import time

2024-04-02 12:10:17,941 WARN util.Utils: Your hostname, BDS-2023 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
2024-04-02 12:10:17,949 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-04-02 12:11:05,863 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.2.3
      /_/

Using Python version 3.10.12 (main, Nov 20 2023 15:14:05)
Spark context Web UI available at http://10.0.2.15:4040
Spark context available as 'sc' (master = local[*], app id = local-1712056298947).
SparkSession available as 'spark'.


In [3]:
from pyspark.sql import SparkSession, SQLContext
#data input

spark_yog = SparkSession.builder.appName('RandomFlorest').getOrCreate()
df_yog = spark_yog.read.csv('hdfs://localhost:9000/user/hduser/Data_Combined.csv', header = True, inferSchema = True)

In [4]:
# processing data

cloum_set=df_yog.columns
categorical_Columns = ['proto', 'service', 'state']
stages_feat = []


In [5]:
# changes the data into vectors that library can undestand and saved in stages_feat

for categorical_Col in categorical_Columns:
    stringIndexer = StringIndexer(inputCol = categorical_Col, outputCol = categorical_Col + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categorical_Col + "classVec"])
    stages_feat += [stringIndexer, encoder]

In [6]:
# Change/Feed Label
Label_str_Indexer = StringIndexer(inputCol = 'attack_cat', outputCol = 'Pred_Label')
stages_feat += [Label_str_Indexer]

In [7]:
#all labels used
numeric_Cols=['dur','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl'	,'ct_dst_ltm',	'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports']
# Random 5 labels
# numeric_Cols= ['sttl', 'sbytes', 'dwin', 'smean', 'dmean']
# Another set of random 5 labels
# numeric_Cols = ['sloss', 'sload', 'stcpb', 'dtcpb', 'trans_depth']
assembler_Inputs = [c + "classVec" for c in categorical_Columns] + numeric_Cols
#labeling the output
Assembler = VectorAssembler(inputCols=assembler_Inputs, outputCol="features_all")
stages_feat += [Assembler]

In [8]:
# Pipeline processing of data to take into the library
pipeline = Pipeline(stages = stages_feat)
pipelineModel = pipeline.fit(df_yog)
df_yog = pipelineModel.transform(df_yog)
selected_Cols = ['Pred_Label', 'features_all'] + cloum_set
df_yog = df_yog.select(selected_Cols)
# df_yog.printSchema()

In [9]:
# splits = df_yog.randomSplit([0.6,0.4], 1234)
training_data, testing_data = df_yog.randomSplit([0.6805,0.3195], seed = 99999999)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(testing_data.count()))

2024-04-02 12:22:56,425 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Training Dataset Count: 112371


Test Dataset Count: 52293


In [10]:
#Model
RandomForest = RandomForestClassifier(labelCol="Pred_Label", featuresCol="features_all", numTrees=10)

start=time.time()
RandomForestModel = RandomForest.fit(training_data)
end=time.time()
start1=time.time()
f_predictions = RandomForestModel.transform(testing_data)
end1=time.time()

In [13]:
# #PRINT CONFUSION MATRIX
#Cm=f_predictions.select("PoutLabel","label").distinct().toPandas()
#f_predictions.groupBy("PoutLabel","prediction").count().show()

AnalysisException: cannot resolve 'PoutLabel' given input columns: [Pred_Label, ackdat, attack_cat, ct_dst_ltm, ct_dst_sport_ltm, ct_dst_src_ltm, ct_flw_http_mthd, ct_ftp_cmd, ct_src_dport_ltm, ct_src_ltm, ct_srv_dst, ct_srv_src, ct_state_ttl, dbytes, dinpkt, djit, dload, dloss, dmean, dpkts, dtcpb, dttl, dur, dwin, features_all, id, is_ftp_login, is_sm_ips_ports, label, prediction, probability, proto, rate, rawPrediction, response_body_len, sbytes, service, sinpkt, sjit, sload, sloss, smean, spkts, state, stcpb, sttl, swin, synack, tcprtt, trans_depth];
'Aggregate ['PoutLabel, prediction#1920], ['PoutLabel, prediction#1920, count(1) AS count#2075L]
+- Project [Pred_Label#1019, features_all#1104, id#16, dur#17, proto#18, service#19, state#20, spkts#21, dpkts#22, sbytes#23, dbytes#24, rate#25, sttl#26, dttl#27, sload#28, dload#29, sloss#30, dloss#31, sinpkt#32, dinpkt#33, sjit#34, djit#35, swin#36, stcpb#37L, ... 26 more fields]
   +- Project [Pred_Label#1019, features_all#1104, id#16, 

In [14]:
print("Time to train:")
print(end-start)
print("Time to predict:")
print(end1-start1)

Time to train:
536.9380197525024
Time to predict:
3.258512020111084


In [15]:
evaluu = BinaryClassificationEvaluator(labelCol="Pred_Label",rawPredictionCol="prediction")
print("Accuracy of model")
print(evaluu.evaluate(f_predictions))
# print(evaluu.explainParams())

Accuracy of model


0.8473149053503936


In [16]:
# #MULT CLASS
evaluator =MulticlassClassificationEvaluator(labelCol="Pred_Label",predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(f_predictions)
print("ACCURACY :")
print(accuracy)

ACCURACY :
0.7581129405465359


In [17]:
evaluator =MulticlassClassificationEvaluator(labelCol="Pred_Label",predictionCol="prediction", metricName="f1")
score= evaluator.evaluate(f_predictions)
print("F1-score")
print(score)

F1-score
0.6830016540293715


In [19]:
spark_yog.stop()